# INTELIGENCIA ARTIFICIAL: EB Ejercicios Tema 5: REDES NEURONALES 
## EJERCICIO: ¿Cómo calcular el número óptimo de neuronas en la capa oculta?
**Implementar** una red neuronal para clasificar emails en SPAM/no SPAM con una única capa oculta. El número de neuronas de la capa oculta se debe calcular de forma óptima con una búsqueda grid usando para ello un conjunto de validación. Use como método de evaluación holdout, donde el conjunto de entrenamiento está formado por el 70% de las primeras filas del conjunto de datos, y el conjunto de test por el resto. 

Después de un preprocesado de los emails, cada email se codifica como un vector de 1899 elementos, donde cada elemento es una característica concreta del email. Todos los emails se encuentran repartidos en los ficheros **spamTrain.mat**, **spamVal.mat** y **spamTest.mat**, en los que tenéis la matriz de atributos **X** y el vector de clase **y** correspondiente al conjunto de training, de validación y de test, respectivamente. 

In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.optimize as opt

# A) DECLARACIÓN DE FUNCIONES Y CARGA DE DATOS

## 1) Cargar los datos de entrada
Estos datos están almacenados en ficheros spamTrain.mat, spamVal.mat y spamTest.mat. 
Visualizar las dimensiones de la matriz de atributos y del vector clase para cada conjunto.

In [2]:
print("Loading Data ...\n")

# Datos de entrenamiento
data_train = sio.loadmat("spamTrain.mat") 
X_train = data_train['X']
y_train = data_train['y']
print("El tamaño de X_train es: ", X_train.shape[0], " filas y ", X_train.shape[1], " columnas.")
print("El tamaño de y_train es: ", y_train.shape[0], " filas y ", y_train.shape[1], " columnas. ")

# Datos de validación
data_val = sio.loadmat("spamValidation.mat") 
X_val = data_val['X']
y_val = data_val['y']
print("El tamaño de X_val es: ", X_val.shape[0], " filas y ", X_val.shape[1], " columnas.")
print("El tamaño de y_val es: ", y_val.shape[0], " filas y ", y_val.shape[1], " columnas. ")

# Datos de test
data_test = sio.loadmat("spamTest.mat") 
X_test = data_test['Xtest']
y_test = data_test['ytest']
print("El tamaño de X_test es: ", X_test.shape[0], " filas y ", X_test.shape[1], " columnas.")
print("El tamaño de y_test es: ", y_test.shape[0], " filas y ", y_test.shape[1], " columnas. ")

Loading Data ...

El tamaño de X_train es:  3000  filas y  1899  columnas.
El tamaño de y_train es:  3000  filas y  1  columnas. 
El tamaño de X_val es:  1000  filas y  1899  columnas.
El tamaño de y_val es:  1000  filas y  1  columnas. 
El tamaño de X_test es:  1000  filas y  1899  columnas.
El tamaño de y_test es:  1000  filas y  1  columnas. 


## 2) Hipótesis y función coste
El objetivo de la red neuronal es minimizar la función de coste definida por: 
\begin{equation}
J(\Theta)=\frac{-1}{m}\sum_{i=1}^{m} \sum_{k=1}^{K} (y_k^i \cdot log((h_{\theta}(x^i))_k)+(1-y_k^i)\cdot log(1-(h_{\theta}(x^i))_k))
\end{equation}

Con el objeto de usar funciones de optimización avanzada de Python, implementar una función que devuelva el valor del coste.


In [3]:
# Función sigmoide tradicional
def sigmoid(z):
  g = 1 / (1+np.exp(-z))
  return g

# Función propagación hacia delante o forward propagation
def forward(theta1, theta2, X, i):
  # bias  + neuronas de la capa 1
  a1 = np.hstack((1, X[i])) # Igual que: ones = np.ones(1)  a1 = np.hstack((ones, X[i]))
  a2 = sigmoid(theta1 @ a1 )

  # bias + neuronas de la capa 2
  a2 = np.hstack((1, a2)) # Igual que: a2 = np.hstack((ones, a2))
  
  # a3 es la salida de la capa 3 o hipótesis (h)
  a3 = sigmoid(theta2 @ a2)
  return a1, a2, a3

# Función coste sin regularizar para redes neuronales
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
  # Paso 1: Enrollar nn_params para obtener cada uno de los theta (pesos/parámetros)
  theta1 = np.reshape(a = nn_params[:hidden_layer_size*(input_layer_size+1)], # datos 
                      newshape = (hidden_layer_size, input_layer_size+1), # nueva shape: (hidden_layer_size, input_layer_size+1)
                      order = 'F') # El order debe ser el mismo que cuando desenrollemos
  theta2 = np.reshape(a = nn_params[hidden_layer_size*(input_layer_size+1):],
                      newshape = (num_labels, hidden_layer_size+1), 
                      order = 'F')
  
  # Paso 2: Definir variables necesarias
  m = len(y) 
  suma = 0 
  y_d = pd.DataFrame(y) # ¡¡IMPORTANTE!!: No aplicamos one-hot encoding (y_d = pd.get_dummies(y.flatten())) ya que solo tenemos 1 clase: spam/no spam. 
  # Pero es importante transformar y a DataFrame para poder acceder fila por fila

  # Paso 3: Para cada fila 
  for i in range(X.shape[0]):
      # Paso 3.1: Forward propagation
      a1, a2, h = forward(theta1, theta2, X, i)
      # Paso 3.2: Coste (como en regresión logística)
      temp1 = y_d.iloc[i] * (np.log(h)) 
      temp2 = (1 - y_d.iloc[i]) * np.log(1 - h) 
      temp3 = np.sum(temp1 + temp2) 
      suma = suma + temp3
  J = (np.sum(suma) / (-m))
  return J

## 3) Gradiente
Para poder usar funciones de optimización avanzadas de Python, implementar una función que devuelva el valor del gradiente formado por las derivadas parciales. 
Recordar que las derivadas parciales se calculan con el algoritmo de Backpropagation.

In [4]:
def nnGradFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
  # Paso 1: Enrollar nn_params para obtener cada uno de los theta (pesos/parámetros)
  initial_theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                              (hidden_layer_size, input_layer_size + 1), 'F')
  initial_theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                              (num_labels, hidden_layer_size + 1), 'F')
  
  # Paso 2: Definir variables necesarias
  m = len(y)
  y_d = pd.DataFrame(y) # ¡¡IMPORTANTE!!: No aplicamos one-hot encoding (y_d = pd.get_dummies(y.flatten())) ya que solo tenemos 1 clase: spam/no spam. 
  # Pero es importante transformar y a DataFrame para poder acceder fila por fila
  delta1 = np.zeros(initial_theta1.shape) # Delta1 tendrá las mismas dimensiones que initial_theta1
  delta2 = np.zeros(initial_theta2.shape) # Delta2 tendrá las mismas dimensiones que initial_theta2

  # Paso 3: Para cada fila
  for i in range(X.shape[0]):
      # Paso 3.1: Forward propagation
      a1, a2, a3 = forward(initial_theta1, initial_theta2, X, i)
      # Paso 3.2: Cálculo de los delta/errores (capa 1 no tiene)
      d3 = a3 - y_d.iloc[i] # última capa 
      d2 = np.multiply(np.dot(initial_theta2.T,d3), np.multiply(a2, 1-a2)) # capa 2
      # Paso 3.3: Cálculo de las derivadas ajustando las dimensiones de los errores y las activaciones de cada capa correctamente
      delta1 = delta1 + (np.reshape(d2[1:,],(hidden_layer_size, 1)) @ np.reshape(a1, (1, input_layer_size+1))) # IGUAL: delta1 = delta1 + d2[1:,np.newaxis] @ a1[np.newaxis, :] 
      delta2 = delta2 + (np.reshape(d3.values, (num_labels, 1)) @ np.reshape(a2, (1, hidden_layer_size+1))) # IGUAL: delta2 = delta2 + d3[:,np.newaxis] @ a2[np.newaxis, :] 

  # Paso 4: Se desenrollan ambas derivadas con el mismo order con el que se enrollaron
  delta1 /= m
  delta2 /= m
  gradiente = np.hstack((delta1.ravel(order='F'), delta2.ravel(order='F')))
  return gradiente

## 4) Función para entrenar la red neuronal usando optimizador avanzado
Implementar una función llamada training que sea la encargada de realizar el entrenamiento de una red neuronal. Para ello, esta función recibirá los parámetros theta iniciales, el tamaño de la capa de entrada, el tamaño de la capa oculta, el número de clases y el conjunto con el que se realice el entrenamiento (X e y). Devolverá los parámetros theta óptimos que definen el modelo aprendido.  

Se recomienda el uso de la función fmin_cg de la librería scipy.optimize. Esta función tiene como argumentos de salida el vector nn_params que contiene los pesos óptimos Theta1 y Theta 2 en forma de vector y el valor J alcanzado. 

Recordar que a partir del vector nn_params hay que reconstruir los pesos Theta1 y Theta2 con las dimensiones adecuadas, usando la función de Python reshape.

In [5]:
def training(initial_theta1, initial_theta2, X_train, y_train, input_layer_size, hidden_layer_size, num_labels):
  maxiter = 10 # Si tarda demasiado, se puede bajar el número de iteraciones al hacer la prueba inicial para comprobar que el entrenamiento es el adecuado

  # Paso 1: Desenrollar los parámetros con el mismo order con el que se enrollaron
  nn_initial_params = np.hstack((initial_theta1.ravel(order='F'), initial_theta2.ravel(order='F')))

  # Paso 2: Llamada al optimizador avanzado gradiente conjugado con la función: fmin_cg
  nn_params = opt.fmin_cg(maxiter=maxiter, f=nnCostFunction, x0=nn_initial_params, fprime=nnGradFunction,
                        args=(input_layer_size, hidden_layer_size, num_labels, X_train, y_train.flatten()), gtol = 0.005)
  
  # Paso 3: Enrollar los pesos/parámetros theta1 y theta2 desde la salida del optimizador avanzado (nn_params)
  theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                      (hidden_layer_size, input_layer_size + 1), order = 'F')
  theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):], 
                      (num_labels, hidden_layer_size + 1), order = 'F')
  return theta1, theta2

## 5) Función para predecir
Implementar una función para predecir si un conjunto de emails son SPAM/no SPAM (binario).

Podrías usar directamente la función forward (fila por fila) y añadir la última línea que calcula la predicción {0,1}.

In [6]:
# COMPLETAR
def predict(theta1, theta2, X):
  # Variables útiles
  m = len(X)
  ones = np.ones((m,1))

  a1 = np.hstack((ones, X))
  a2 = sigmoid(a1 @ theta1.T)
  a2 = np.hstack((ones, a2))
  h = sigmoid(a2 @ theta2.T) # La hipótesis o predicción 

  # Si h es mayor o igual que 0.5, entonces la predicción será 1, si es menor que 0.5 será 0
  # Función np.where: https://numpy.org/doc/stable/reference/generated/numpy.where.html : np.where(condicion, value if TRUE, value if FALSE)
  pred = np.where( h >= 0.5 , 1 , 0)
  return pred

## 6) Cálculo del número óptimo de neuronas de la capa oculta con búsqueda grid usando el conjunto de validación.
La función optimalHiddenNeurons se encarga de calcular el número óptimo de neuronas en la capa oculta con una búsqueda grid usando un conjunto de validación. 

Para ello, esta función recibirá el tamaño de la capa de entrada, el conjunto con el que se realice el entrenamiento, y el conjunto con el que se calcule la tasa de acierto, y devolverá el número óptimo de neuronas. El número óptimo de neuronas será aquel que haga máxima la tasa de acierto. 

El grid estará formado por el conjunto de neuronas {1,2,3,4,5,6,7,8,9,10}.

En esta función se imprimirá la tasa de acierto para el grid de neuronas, el número óptimo de neuronas y la tasa de acierto máxima alcanzada.

### 6.1) Función para inicializar los pesos aleatoriamente de una capa 
Esta función recibe como parámetro el peso de una capa con L_in neuronas de entrada y L_out neuronas de salida. [EB TEMA 5 PARTE II ÚLTIMA DIAPOSITIVA]

In [7]:
# COMPLETAR
def randInitializeWeights(L_in, L_out):
  # Variable a devolver tendrá dimensiones: (L_out, L_in+1) # +1 procedente de la bias
  W = np.zeros((L_out, 1+L_in))

  # Se va a inicializar W de manera random para "romper" la simetría mientras se entrena la red neuronal
  epsilon_init = 0.12 # Se define un epsilon
  W = np.random.rand(L_out, L_in+1) * 2 *  epsilon_init - epsilon_init
  return W

In [8]:
# COMPLETAR
def optimalHiddenNeurons(input_layer_size, num_labels, X_train, y_train, X_val, y_val):
  # Paso 1: # Inicializamos variables útiles
  num_max_neuronas = 3 # el número máximo de neuronas en el grid
  print('\nCalculando número óptimo de neuronas de la capa oculta... \n')
  print('\nNúmero máximo de neuronas: ',num_max_neuronas)

  arr_accuracy = [] # Inicializamos la lista donde almacenaremos la precisión del conjunto de 
  # validación para los diferentes números de neuronas del grid

  # Paso 2: Bucle desde 1 hasta num_max_neuronas (incluido, por eso el +1)
  for hidden_layer_size in range(1, num_max_neuronas+1):
    print('-----\nNúmero de neuronas de la capa oculta: ',hidden_layer_size)

    # Paso 2.1: Inicializar los pesos aleatoriamente con las dimensiones correctas
    initial_theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
    initial_theta2 = randInitializeWeights(hidden_layer_size, num_labels)

    # Paso 2.2: Entrenamiento de la red neuronal con el número de neuronas de la capa oculta
    theta1_opt, theta2_opt = training(initial_theta1, initial_theta2, X_train, y_train, input_layer_size, hidden_layer_size, num_labels)

    # Paso 2.3: Predicción usando el conjunto de validación
    pred = predict(theta1_opt , theta2_opt, X_val)
    
    # Paso 2.4: Calcular la precisión/accuracy máxima
    arr_accuracy.append(np.mean(pred== y_val )) # Se añade a la lista de precisión
    print("accuracy: ", np.mean(pred== y_val ))
    
  # Paso 3: Fuera del bucle, encontrar el número de neuronas ocultas con las que se consigue el mejor accuracy
  optimal_hidden_layer_size = np.argmax(arr_accuracy)+1 # +1 porque np.argmax() nos proporciona la posición en la lista del mejor valor. Las posiciones empiezan en 0 y nosotros empezamos en 1 neurona
  print("\n**** El número de neuronas de la capa oculta óptimo es: ", optimal_hidden_layer_size)
  print("**** Con esas neuronas en la capa oculta, el accuracy del conjunto de validación es: ", max(arr_accuracy))

  return optimal_hidden_layer_size

# B) EJECUCIÓN DEL PROBLEMA BUSCANDO EL NÚMERO ÓPTIMO DE NEURONAS DE LA CAPA OCULTA Y PREDICIENDO SOBRE EL CONJUNTO DE TEST

In [9]:
# Paso 1: Inicializar el tamaño de la capa de entrada y el número de clases del problema
input_layer_size = 1899 # Número de columnas de los datos de entrada (X)
num_labels = 1 # Spam/No spam

# Paso 2: Calcular el número óptimo de neuronas en la capa oculta usando la función optimalHiddenNeurons
optimal_hidden_layer_size = optimalHiddenNeurons(input_layer_size, num_labels, X_train, y_train, X_val, y_val)

# Paso 3: Inicialización de pesos con el número óptimo de neuronas en la capa oculta
initial_theta1 = randInitializeWeights(input_layer_size, optimal_hidden_layer_size)
initial_theta2 = randInitializeWeights(optimal_hidden_layer_size, num_labels)

# Paso 4: Entrenamiento de la red neuronal con el número de neuronas óptimo y obtener los pesos óptimos. 
# Recordar que el entrenamiento debe hacerse con el conjunto de training entero (X_train y X_val)
X_train_completo = np.append(X_train, X_val) # IMPORTANTE
y_train_completo = np.append(y_train, y_val) # IMPORTANTE
theta1_opt, theta2_opt = training(initial_theta1, initial_theta2, X_train_completo, y_train_completo, input_layer_size, optimal_hidden_layer_size, num_labels)

# Paso 5: Predecir usando el conjunto de test y calcular el error
pred = predict(theta1_opt, theta2_opt, X_test )
print("Accuracy del conjunto de test: ", np.mean(pred == y_test )) # Calcular la precisión/accuracy máxima


Calculando número óptimo de neuronas de la capa oculta... 


Número máximo de neuronas:  3
-----
Número de neuronas de la capa oculta:  1
Optimization terminated successfully.
         Current function value: 0.140893
         Iterations: 7
         Function evaluations: 22
         Gradient evaluations: 22
accuracy:  0.953
-----
Número de neuronas de la capa oculta:  2
Optimization terminated successfully.
         Current function value: 0.098913
         Iterations: 8
         Function evaluations: 28
         Gradient evaluations: 28
accuracy:  0.975
-----
Número de neuronas de la capa oculta:  3
Optimization terminated successfully.
         Current function value: 0.110434
         Iterations: 6
         Function evaluations: 20
         Gradient evaluations: 20
accuracy:  0.967

**** El número de neuronas de la capa oculta óptimo es:  2
**** Con esas neuronas en la capa oculta, el accuracy del conjunto de validación es:  0.975


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1900)